In [ ]:
#| default_exp trial_components
%load_ext autoreload
%autoreload 2

# trial components

> All classes that serve as components for instantiating a full trial.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import time
import datetime
from typing import List
from pathlib import Path
import warnings
import itertools as it

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
import numpy as np
import speech_recognition as sr
import sounddevice as sd

from task_tracker.utils import get_duration_in_s_from_timestamps, transcribe_audio_to_task, create_cumulative_bar_plots, create_timeline, create_cumulative_tie_plots, create_cumulative_dataframe, save_trial, create_cumulative_pie_plots_per_lane, create_cumulative_bar_plots_per_lane, get_colors, create_histplot

In [ ]:
#| export
class Proband():
    """
    Stores metadata associated with the proband.
    """
    def __init__(self, proband_ID):
        self.metadata = {}
        self.proband_ID = proband_ID
    
    def set_metadata(self, metadata, metadata_name):
        self.metadata[metadata_name]=metadata

In [ ]:
#| export
class Trial():
    """
    The main class used by task_tracker.
    `Proband`, `Task_History`, `Audio_Record` objects are stored here. 
    Metadata such as user_ID, start_ and end_time are stored here.
    `Main_Interface` writes acquired information directly to this object.
    """
    def __init__(self, user_ID, task_dict, demographic_dict, target_dir = None, proband_ID = None, colors = None, language="german", model="medium", descriptions_preset=[]):
        self.language = language
        self.task_dict = task_dict
        self.model = model
        self.demographic_dict = demographic_dict
        self.user_ID = user_ID
        self.task_number = 0
        self.start_time = None
        self.out_dir = None
        self.started = False
        self.colors = colors
        self.descriptions_preset = descriptions_preset
        if target_dir is None or not target_dir.exists():
            self.target_dir = Path.cwd()
        else:
            self.target_dir = target_dir
        if proband_ID is None:
            proband_ID = self._get_next_proband_ID()
        self.history = Task_History(task_dict)
        self.proband = Proband(proband_ID)
        self.audio_record = Audio_Record(trial=self)
    
    def set_start_time(self, overwrite=False):
        if self.start_time is None and not overwrite:
            self.start_time = round(time.time(), 4)
            self.start_struct_time = time.gmtime()
        self.out_dir = self.target_dir.joinpath(f"{time.strftime('%Y-%m-%d_%H.%M.%S', self.start_struct_time)}_{self.proband.proband_ID}_{self.user_ID}/")
        self.started = True
        Path.mkdir(self.out_dir)
        
    def end_trial(self):
        self.end_time = round(time.time(), 4)
        self.end_struct_time = time.gmtime()
        
        save_trial(self)
        
    def export_results(self):
        """
        Wrapper function. Executes all export options.
        """
        self.export_tasks()
        self.export_tasks_per_subtasks()
        self.export_proband_information()

        save_trial(self)
        
        return self.tasks_dataframe, self.cum_d, self.dataframe_per_subtasks, self.plots, self.subtask_plots, self.metadata_df
    
    def export_problems(self):
        self.history.export_problems()
        self.history.category_dataframe.to_excel(self.out_dir.joinpath("problems_categories.xlsx"))
        fig = create_histplot(self.history.category_dataframe)
        fig.savefig(self.out_dir.joinpath("histplot_categories_tasks.png"))
        return self.history.category_dataframe, fig
        
    def export_tasks(self):
        """
        Creates the dataframe with information per single task, the full cumulative dataframe and the timeline and the full cumulative plots and timeline.
        """
        self._postprocess_tasks_and_descriptions()
        self.tasks_dataframe = self.history.export_tasks()
        self.tasks_dataframe.to_excel(self.out_dir.joinpath(f"tasks.xlsx"))

        task_names = self.tasks_dataframe["task_name"].unique()
        if self.colors is None:
            self.colors = get_colors(task_names)
            
        for task_name in task_names:
            if task_name not in self.colors:
                self.colors[task_name] = sns.colors.xkcd_rgb["black"]
        
        self.plots = {}
        for func, name in [(create_timeline, "timeline"), (create_cumulative_bar_plots, "cumulative_bar_plots"), (create_cumulative_tie_plots, "cumulative_tie_plots")]:
            fig = func(self.tasks_dataframe, task_names = task_names, colors = self.colors)
            self.plots[name] = fig
            fig.savefig(self.out_dir.joinpath(f"{name}.png"))
        
        self.cum_d = create_cumulative_dataframe(self.tasks_dataframe)
        self.cum_d.to_excel(self.out_dir.joinpath(f"tasks_cumulative_duration.xlsx"))
        
        save_trial(self)
        
        return self.plots, self.tasks_dataframe, self.cum_d
    
    def export_proband_information(self):
        """
        Creates the dataframe with the proband metadata.
        """
        self.metadata_df = pd.DataFrame(self.proband.metadata, columns=self.proband.metadata.keys(), index=[self.proband.proband_ID])
        self.metadata_df.to_excel(self.out_dir.joinpath(f"proband_metadata.xlsx"))
        
        save_trial(self)
        
        return self.metadata_df
        
    def export_tasks_per_subtasks(self):
        """
        Creates dataframe and plots for subtasks per tasks.
        """
        tasks = {}
        for lane in self.history.tasks:
            for task in self.history.tasks[lane]:
                if task.task_name != "Pause":
                    if task.task_name not in tasks:
                        tasks[task.task_name] = {}
                    end = task.end_time
                    start = task.start_time
                    
                    for sub_lane in self.history.tasks:
                        if sub_lane != lane:
                            for subtask in self.history.tasks[sub_lane]:
                                
                                if subtask.start_time < end and subtask.end_time > start:
                                    duration = subtask.calculate_duration(start, end)
                                    if subtask.task_name not in tasks[task.task_name]:
                                        tasks[task.task_name][subtask.task_name] = duration
                                    else:
                                        tasks[task.task_name][subtask.task_name] += duration

        self.dataframe_per_subtasks = pd.DataFrame(tasks)
        self.dataframe_per_subtasks.to_excel(self.out_dir.joinpath(f"subtasks_per_tasks.xlsx"))

        self.subtask_plots = {}
        for func, name in [(create_cumulative_bar_plots_per_lane, "bar_plots_per_lane"), (create_cumulative_pie_plots_per_lane, "pie_plots_per_lane")]:
            fig = func(self.dataframe_per_subtasks)
            self.subtask_plots[name] = fig
            fig.savefig(self.out_dir.joinpath(f"{name}.png"))
            
        save_trial(self)
            
        return self.subtask_plots, self.dataframe_per_subtasks
        
        
    def _get_next_proband_ID(self):
        probands = []
        for file in self.target_dir.iterdir():
            if file.is_dir():
                try:
                    timestemp_day, timestemp_time, old_proband_ID, user_ID = file.name.split("_")
                    probands.append(old_proband_ID)
                except ValueError:
                    pass
        if probands:
            probands.sort()
            proband_ID = str(int(probands[-1])+1).zfill(4)
        else:
            proband_ID = "0000"
        return proband_ID
        
    def _postprocess_tasks_and_descriptions(self):
        self.audio_record.transcribe_audio()

        for lane in self.history.tasks:
            for task in self.history.tasks[lane]:
                if type(task)==Task:
                    for segment in self.audio_record.transcription["segments"]:
                        if (segment["start"] > task.start_time and segment["start"] < task.end_time) or (segment["start"] < task.start_time and segment["end"] > task.start_time):
                            task.add_description(segment["text"], segment["start"])
                    

In [ ]:
#| export
class Task():
    """
    The syllables of task_tracker.
    Start-, end- time and `Pause` objects are stored here. Descriptions can be added to this object.
    The duration can be calculated substracting the breaks.
    """
    def __init__(self, task_number, task_name, lane, trial_start_time):
        self.task_name = task_name
        self.task_number = task_number
        self.lane = lane
        self.start_time = None
        self.trial_start_time = trial_start_time
        self.end_time = None
        self.paused = False
        self.running = False
        self.currently_paused = False
        self.pauses = []
        self.pause = None
        self.description = {}
        self.categories = {"start_time": [], "text": [], "categories": [], "task": [], "proband": []}
    
    def start(self):
        if not self.running:
            self.running = True
            self.start_time = round(time.time(), 4)-self.trial_start_time
    
    def pause_start(self):
        if not self.currently_paused:
            self.paused = True
            self.currently_paused = True
            self.pause = Pause(self.task_number, lane=self.lane, trial_start_time=self.trial_start_time)
            self.pause.start()
    
    def end(self):
        if self.running:
            self.end_time = round(time.time(), 4)-self.trial_start_time
            self.pause_end()
            self.running = False
            self.duration_in_s = self.calculate_duration()
            
    def calculate_duration(self, start_time=None, end_time=None):
        if start_time is None: 
            start_time = self.start_time
        elif start_time < self.start_time:
            start_time = self.start_time
        if end_time is None:
            end_time = self.end_time
        elif end_time > self.end_time:
            end_time = self.end_time
        
        if self.paused is False:
            duration = get_duration_in_s_from_timestamps(start_time, end_time)
        else:
            duration = get_duration_in_s_from_timestamps(start_time, end_time)
            for pause in self.pauses:
                if pause.start_time < end_time and pause.end_time > start_time:
                    duration -= pause.calculate_duration(start_time, end_time)
        
        return duration
    
    def pause_end(self):
        if self.currently_paused:
            self.currently_paused = False
            self.pause.end()
            self.pauses.append(self.pause)
            self.pause = None
            
    def add_description(self, description, start_time):
        self.description[start_time] = description

In [ ]:
#| export
class Pause():
    """
    Stores start- and end- time. 
    The duration can be calculated substracting the breaks.
    """
    def __init__(self, task_number, task_name="Pause", lane="Tasks", trial_start_time=0):
        self.task_name = task_name
        self.task_number = task_number
        self.start_time = None
        self.trial_start_time = trial_start_time
        self.end_time = None
        self.lane = lane
        self.description = {}
    
    def start(self):
        self.start_time = round(time.time(), 4)-self.trial_start_time
    
    def end(self):
        self.end_time = round(time.time(), 4)-self.trial_start_time
        self.duration_in_s = self.calculate_duration()
        
    def calculate_duration(self, start_time=None, end_time=None):
        if start_time is None: 
            start_time = self.start_time
        elif start_time < self.start_time:
            start_time = self.start_time
        if end_time is None:
            end_time = self.end_time
        elif end_time > self.end_time:
            end_time = self.end_time
        
        duration = get_duration_in_s_from_timestamps(start_time, end_time)
        
        return duration

In [ ]:
#| export
class Audio_Record():
    """
    Audio is recorded via this class.
    The interaction with sounddevice is performed here.
    Per default, the recording can last no longer than 3 hours. 
    However, the duration can be set individually.
    """
    def __init__(self, trial, channels = 1, freq = 44100, duration = 10800):
        self.channels = channels
        self.freq = freq
        self.duration = duration
        self.trial = trial
        self.filename = None
        self.recording = np.array([])
        self.running = False
    
    def start(self):
        if not self.running:
            import sounddevice as sd
            self.start_time=round(time.time(), 4)
            self.recording = sd.rec(int(self.duration * self.freq), 
                       samplerate=self.freq, channels=self.channels)
            self.running = True
            self.filename = str(self.trial.out_dir.joinpath("recording.WAV"))
        
    def end(self):
        if self.running:
            import sounddevice as sd
            self.end_time=round(time.time(), 4)
            sd.stop()
            real_duration = get_duration_in_s_from_timestamps(self.start_time, self.end_time)
            self.recording = self.recording[:int(real_duration*self.freq)]
            y = (np.iinfo(np.int32).max * (self.recording/np.abs(self.recording).max())).astype(np.int32)
            write(self.filename, self.freq, y)
            self.running = False
            
    def transcribe_audio(self):
        r = sr.Recognizer()
        file = sr.AudioFile(self.filename)
        with file as source:
            audio = r.record(source)
        self.transcription = r.recognize_whisper(audio, show_dict=True, language=self.trial.language, model=self.trial.model)
        return self.transcription

In [ ]:
#| export
class Task_History():
    """
    The tasks are stored here. 
    They can be exported as dataframe via `Task_History.export_tasks`.
    """
    def __init__(self, task_dict):
        self.tasks = self._create_tasks_dict(task_dict)
        self.current_tasks = {}
        self.dataframe = None
        
    def add_current_task_to_history(self, lane):
        self.tasks[lane].append(self.current_tasks[lane])
        
    def add_pause(self, pause):
        self.tasks["Pause"].append(pause)
        
    def export_tasks(self):
        """
        Creates a dataframe from all stored tasks.
        """
        self.dataframe = pd.DataFrame()
        tasks = []
        for lane in self.tasks:
            tasks += self.tasks[lane]
        self.dataframe["lane"] = [task.lane for task in tasks]
        self.dataframe["duration_in_s"] = [task.duration_in_s for task in tasks]
        self.dataframe["task_number"] = [task.task_number for task in tasks]
        self.dataframe["task_name"] = [task.task_name for task in tasks]
        self.dataframe["start_time"] = [task.start_time for task in tasks]
        self.dataframe["end_time"] = [task.end_time for task in tasks]
        with warnings.catch_warnings():
            warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
            for lane in self.tasks:
                for task in self.tasks[lane]:
                    for i, start_time in enumerate(task.description):
                        self.dataframe.loc[self.dataframe["task_number"]==task.task_number, f"description_{i}"] = str(round(start_time, 1)) + " " + task.description[start_time]
        return self.dataframe
    
    
    def export_problems(self):
        """
        Creates a dataframe from all coded categories.
        """
        self.category_dataframe = pd.DataFrame()
        tasks = []
        for lane in self.tasks:
            for task in self.tasks[lane]:
                if type(task) == Task and hasattr(task, "categories"):
                    tasks.append(task)
        self.category_dataframe["task"] = list(it.chain(*[task.categories["task"] for task in tasks]))
        self.category_dataframe["start_time"] = list(it.chain(*[task.categories["start_time"] for task in tasks]))
        self.category_dataframe["category"] = list(it.chain(*[task.categories["categories"] for task in tasks]))
        self.category_dataframe["text"] = list(it.chain(*[task.categories["text"] for task in tasks]))
        self.category_dataframe["proband"] = list(it.chain(*[task.categories["proband"] for task in tasks]))
        return self.category_dataframe
    
    
    def _create_tasks_dict(self, task_dict):
        tasks = {"Pause": []}
        if type(task_dict) == dict:
            for lane in task_dict:
                tasks[lane] = []
        elif type(task_dict) == list:
            tasks["Tasks"] = []
        return tasks

In [ ]:
#| export
class Segment():
    """
    Representations of single segments transcribed from audio. Are used in `Correct_Transcription_Interface`.    
    """
    def __init__(self, start_time, end_time, text, ide, array_slice, tasks, trial):
        self.start_time = start_time
        self.trial = trial
        self.end_time = end_time
        self.id = ide
        self.array_slice = array_slice
        self.text = text
        self.checked = False
        self.tasks = self._find_tasks(tasks)
        
    def _find_tasks(self, all_tasks):
        matching_tasks = []
        for lane in all_tasks:
            for task in all_tasks[lane]:
                if type(task)==Task:
                    if (self.start_time > task.start_time and self.start_time < task.end_time) or (self.start_time < task.start_time and self.end_time > task.start_time):
                        matching_tasks.append(task)
        return matching_tasks
        
    def play_segment(self):
        sd.play(self.array_slice)
        
    def replace_text(self, new_text):
        for task in self.tasks:
            task.description[self.start_time] = new_text
        for segment in self.trial.audio_record.transcription["segments"]:
            if segment["text"] == self.text:
                segment["text"] = new_text
        self.trial.audio_record.transcription["text"]=self.trial.audio_record.transcription["text"].replace(self.text, new_text)
        self.text = new_text
        self.checked = True
        
    def delete_text(self):
        for task in self.tasks:
            if self.start_time in task.description:
                task.description.pop(self.start_time)
        for segment in self.trial.audio_record.transcription["segments"]:
            if segment["text"] == self.text:
                segment["text"] = ""
        self.trial.audio_record.transcription["text"]=self.trial.audio_record.transcription["text"].replace(self.text, "")
        self.text = ""
        self.checked = True
        
    def add_category(self, category):
        for task in self.tasks:
            if not hasattr(task, "categories"):
                task.categories = {"start_time": [], "text": [], "categories": [], "task": [], "proband": []}
            if category != None:
                if self.start_time not in task.categories["start_time"]:
                    # testen, dass mna nur eine kategorie hizufügen kann!
                    task.categories["start_time"].append(self.start_time)
                    task.categories["text"].append(self.text)
                    task.categories["categories"].append(category)
                    task.categories["task"].append(task.task_name)
                    task.categories["proband"].append(self.trial.proband.proband_ID)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()